# New issues in the news

First thing we need to do is to:

+ reference the Thomson Reuters Eikon API; 
+ create an application ID for our script.

In [1]:
import eikon as tr
import pandas as pd

In [2]:
tr.set_app_id('your_app_id')

We are going to request emerging market new issue (ISU) eurobond (EUB) news from International Financial Review Emerging EMEA service (IFREM), focusing on the notifications of the already priced issues.

In [3]:
from datetime import date

start_date, end_date = date(2016, 1, 1), date.today()
q = "Product:IFREM AND Topic:ISU AND Topic:EUB AND (\"PRICED\" OR \"DEAL\")"
headlines = tr.get_news_headlines(query=q, date_from=start_date, date_to=end_date, count=100)
headlines.head()

,versionCreated,text,storyId,sourceCode
2017-04-11 15:03:51,2017-04-11 15:04:33.786,PRICED: X5 RUB20bn 3yr at 9.25%; Leads,urn:newsml:reuters.com:20170411:nIFR3Wb3bN:1,NS:IFR
2017-04-10 20:43:18,2017-04-10 20:48:04.320,PRICED: Romania E1.75bn 2-tranche deal,urn:newsml:reuters.com:20170410:nIFRS9KcK:1,NS:IFR
2017-04-07 10:07:04,2017-04-07 11:29:59.276,DEAL: AFC prices USD500m 3.875% Apr 2024 7yr a...,urn:newsml:reuters.com:20170407:nIFR2Rv27f:1,NS:IFR
2017-04-06 07:54:01,2017-04-06 07:54:02.810,UPDATE: IsDB deal statistics,urn:newsml:reuters.com:20170406:nIFR8fGynv:1,NS:IFR
2017-04-05 21:10:09,2017-04-05 22:59:40.538,PRICED: Petra Diamonds US$650m 5-year nc2 sr s...,urn:newsml:reuters.com:20170405:nIFR5LpzRX:1,NS:IFR


In the context of news, each story has its own unique idenifier, created according to the RFC 3085 standard. Here's what the story looks like, notice that I am using the standard HTML() function from Notebook to display it:

In [4]:
from IPython.core.display import HTML
html = tr.get_news_story('urn:newsml:reuters.com:20170405:nIFR5LpzRX:1')
HTML(html)

We can parse the data using a regular expression, but before this we will need to convert HTML into text. Let's create a function that is going to return a dictionary from the this type of article:

In [5]:
from lxml import html
import re

def termsheet_to_dict(storyId):
    x = tr.get_news_story(storyId)
    story = html.document_fromstring(x).text_content()
    matches = dict(re.findall(pattern=r"\[(.*?)\]:\s?([A-Z,a-z,0-9,\-,\(,\),\+,/,\n,\r,\.,%,\&,>, ]+)", string=story))
    clean_matches = {key.strip(): item.strip() for key, item in matches.items()}
    return clean_matches

Let's test it and see if it works:

In [6]:
termsheet_to_dict('urn:newsml:reuters.com:20170323:nIFR9z7ZFL:1')['NOTES']

'EUR400m (from 300m+) 3yr LPN. RegS. Follows rshow. Exp nr/B+/BB.\r\nAlfa/ING/UBS(B&D). IPTs 2.75% area, guidance 2.625%/2.75% WPIR, set at 2.625% on\r\nbks closed >750m.'

Let's extract all data for all headlines:

In [7]:
result = []

index = pd.DataFrame(headlines, columns=['storyId']).values.tolist()

for i, storyId in enumerate(index):
    x = termsheet_to_dict(storyId[0])
    if x:
        result.append(x)

df = pd.DataFrame(result)
df.head()

,1st Pay,Asset Type,Bookrunners,Business,CUSIP/ISIN,Call,Country,Coupon,DBRS,Denoms,...,Sector,Settledate,Size,Spread,Stabilis,Status,Tenor/Mty,Total,UOP,Yield
0,NaN,High Yield,GS/UBS/VTB,NaN,XS1598697412 Trade House PEREKRIOS...,NaN,RUSSIA,9.250 Fixed,NaN,10m/100k,...,Cons Staples-Retailing Limited Liability...,18-Apr-17,RUB 20BN,NaN,NaN,PRICED,3yr 18-Apr-20,NaN,NaN,9.25
1,NaN,Investment Grade,Barc/Citi/Erste/ING/SG,NaN,XS1313004928 /,NaN,ROMANIA,3.875,NaN,1k+1k,...,NaN,19-Apr-17,EUR 750M,NaN,NaN,PRICED,18.5yr 29-Oct-35,2BN,NaN,3.55\r\nSr Unsec Notes
2,NaN,Investment Grade,Citi/JPM/MUFG/StCh,NaN,144A US00830YAB74 / RegS,NaN,Supranational,3.875 Fixed,NaN,NaN,...,NaN,13-Apr-17,USD 500M,MS+184.3,NaN,PRICED,7yr 13-Apr-24,NaN,NaN,4\r\nSr Unsec Notes
3,NaN,High Yield,Barc/RBC/BMO,NaN,USG7028AAB91 /,NaN,UNITED STATES,7.250 Fixed\r\nTREASURY PLC,NaN,200k/1k,...,Materials-Mining,12-Apr-17,USD 650M,T+540.1bp,NaN,PRICED,5yr 01-May-22,NaN,Mixed,7.25
4,NaN,Islamic,,NaN,XS1597014767,NaN,SAUDI ARABIA,6.875 Fixed,NaN,200k/1k,...,Financials-Real Estate,10-Apr-17,USD 500M,MS+512.8,NaN,PRICED,5yr 10-Apr-22,NaN,Refinance,7.125


Now, when we have the dataframe in place, we can perform simple stats on our data. For instance, how many of those issues reported were Investment Grade versus High Yield.

In [8]:
df['Asset Type'].value_counts()

Investment Grade    50
High Yield          23
Islamic              8
Covered              2
Name: Asset Type, dtype: int64

What about a specific country?

In [9]:
df[df['Country']=='RUSSIA']['Asset Type'].value_counts()

Investment Grade    7
High Yield          6
Name: Asset Type, dtype: int64

You can experiment further by changing the original headline search query, for example, by including the RIC into your request.